In [ ]:
import openai

openai.api_base = 'http://aitools.cs.vt.edu:7860/openai/v1'
openai.api_key = 'aitools'

def get_completion(prompt, model="gpt-3.5-turbo-16k"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message["content"]


def converse(prompt, messages=None, model="gpt-3.5-turbo-16k", max_tokens=4000, temperature=0, top_p=1, frequency_penalty=0,
             presence_penalty=0):
    # Add the user's message to the list of messages
    if messages is None:
        messages = []

    messages.append({"role": "user", "content": prompt})

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
    ).choices[0].message["content"]

    # Add the assistant's message to the list of messages
    messages.append({"role": "assistant", "content": response})

    return response, messages


# Get the OpenAPI specification into the system

In [ ]:
import requests

openapi_json = str(requests.get("http://aitools.cs.vt.edu:8000/openapi.json").json())

messages = []
prompt = f"""
  Using this 
  ```json
  {openapi_json}
  ``` 
OpenAPI specification, output a summary in bullet point form for each endpoint.  Let's make it concise in markdown format.  It should include short descriptions of parameters, and list expected possible response status codes

"""
api_description, messages = converse(prompt, messages)

print(api_description)

In [ ]:
messages = []
prompt = f"""
You are a QA developer expert in Python, using the pytest framework.
You're writing tests for an API, described as:
```
{api_description}
```

The architecture of the test cases is three layers:
1. Core client layer (folder=client) 
2. Test user session layer (folder=session)
3. Test Case layer (folder=test)

Let's use aiohttp for our core client layer.  
Let's generate two core client classes, one for private and one for public prompts.
Let's allow these classes to read the .env for TEST_USERNAME.
Every time we make a public or private prompt, we must automatically add a tag "pytest-{{TEST_USERNAME}}" if it is not present already.
This will let us clean up our database.

Write a complete implementation covering both endpoints.  Use basic authentication where needed.
"""

core_classes, messages = converse(prompt, messages)

print(core_classes)

In [ ]:
prompt = f"""
Summarize the core classes details
```
{core_classes}
```
in bullet point form.
"""
core_layer_classes, messages = converse(prompt, messages)

messages = []
prompt = f"""
You are a QA developer expert in Python, using the pytest framework.

The architecture of the test cases is three layers:
1. Core client layer (folder=client) 
2. Test user session layer (folder=session)
3. Test Case layer (folder=test)

You have core layer classes:
```
{core_layer_classes}
```


Let's use pytest for our test user session layer.
This layer will introduce a class called PublicUserSession and another child class called PrivateUserSession.  They share the same interface/method signatures and differ only in that a user is required for a PrivateUserSession, and is optional for a PublicUserSession.  
Each session will hold a reference to a public and a private core layer client.
The session should take a User object as a parameter.
"""

session_classes, messages = converse(prompt, messages)
print(session_classes)

In [14]:
prompt = f"""
Summarize the session classes details
```
{session_classes}
```
in bullet point form.
"""
session_layer_classes, messages = converse(prompt, messages)

user_class = f"""
class User:
    def __init__(self, username, password):
        self._auth = aiohttp.BasicAuth(username, password)
        self._username = username
        self._password = password
"""

prompt = f"""
You are a QA developer expert in Python, using the pytest framework.

The architecture of the test cases is three layers:
1. Core client layer (folder=client) 
2. Test user session layer (folder=session)
3. Test Case layer (folder=test)

You have these session layer classes:
```
{session_layer_classes}
```

We have a User class that holds a username and password.
```
{user_class}
```

Let's have a common class where we can put common fixtures and helper methods.
Let's call it BaseTest.
In there we should read in our test user from a .env file using the python-dotenv library.
Keys are TEST_USERNAME and TEST_PASSWORD.  
We should also have a fixture that returns a public session without a user (called Alice), and a private session with a user (called Bob) and a public session with a user ( called Adrianna).
Also we need fixture that runs after each test case to delete all prompts that are tagged as "pytest-{{TEST_USERNAME}}".
"""

base_test_class, messages = converse(prompt, messages)
print(base_test_class)

Here's an example implementation of the `BaseTest` class with the common fixtures and helper methods:

```python
import pytest
from dotenv import load_dotenv
from pathlib import Path
from client import PublicPromptClient, PrivatePromptClient
from session import PublicUserSession, PrivateUserSession
from user import User

class BaseTest:
    @pytest.fixture(scope="class")
    def test_user(self):
        env_path = Path('.') / '.env'
        load_dotenv(dotenv_path=env_path)
        username = os.getenv("TEST_USERNAME")
        password = os.getenv("TEST_PASSWORD")
        return User(username, password)

    @pytest.fixture
    def public_session(self):
        session = PublicUserSession()
        yield session
        session.public_client.close()

    @pytest.fixture
    def private_session(self, test_user):
        session = PrivateUserSession(test_user)
        yield session
        session.private_client.close()

    @pytest.fixture
    def public_session_with_user(self, test_use